# Data Science Study Group Project: Neutrino type and background classification
# Building your first fully connected network and a CNN 

As last time launch a job - but now use `jupyter_job_GPU.sh` - this will request a GPU

Also before the meeting please copy the dataset to the local SLURM temporary fast storage

Ok let's make a dataset:

In [1]:
from io_utils.data_handling import WCH5Dataset
dset=WCH5Dataset("/fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5",val_split=0.1,test_split=0.1)

Let's first create a configuration object -we'll use this to set up our training engine

In [2]:
#path = /fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5

In [3]:
class CONFIG:
    pass
config=CONFIG()
config.batch_size_test = 1024
config.batch_size_train = 32
config.batch_size_val = 1024
config.lr=0.001
config.device = 'gpu'
config.gpu_list = [0]
config.dump_path = 'model_state_dumps'


Now Let's make our model. We'll talk about 
  - model parameters
  - inputs and the forward method
  - Modules containing modules
  - Sequential Module
  Lets open [simpleMLP](../edit/models/simpleMLP.py)

In [4]:
from models.simpleMLP import SimpleMLP

In [5]:
model_MLP=SimpleMLP(num_classes=3)

Let's look at the parameters:

In [6]:
print(model_MLP.parameters)

<bound method Module.parameters of SimpleMLP(
  (relu): ReLU()
  (softmax): Softmax()
  (fc1): Linear(in_features=24320, out_features=9728, bias=True)
  (fc2): Linear(in_features=9728, out_features=4864, bias=True)
  (fc3): Linear(in_features=4864, out_features=1600, bias=True)
  (fc4): Linear(in_features=1600, out_features=400, bias=True)
  (fc5): Linear(in_features=400, out_features=3, bias=True)
)>


Now let's look at sequential version

In [7]:
from models.simpleMLP import SimpleMLPSEQ
model_MLPSEQ=SimpleMLPSEQ(num_classes=3)

In [8]:
print(model_MLPSEQ.parameters)

<bound method Module.parameters of SimpleMLPSEQ(
  (_sequence): Sequential(
    (0): Linear(in_features=24320, out_features=9728, bias=True)
    (1): ReLU()
    (2): Linear(in_features=9728, out_features=4864, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4864, out_features=1600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1600, out_features=400, bias=True)
    (7): ReLU()
    (8): Linear(in_features=400, out_features=3, bias=True)
  )
)>


In [9]:
import numpy as np
transform=np.ravel
dset=WCH5Dataset("/fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5",val_split=0.1,test_split=0.1,transform=transform)

In [10]:
from training_utils.engine import Engine

In [11]:
engine=Engine(model_MLP,dset,config)

Requesting GPUs. GPU list : [0]
Main GPU: cuda:0
CUDA is available
Creating a directory for run dump: model_state_dumps/20190619_194628/


In [12]:
print(vars(config))

{'batch_size_test': 1024, 'batch_size_train': 32, 'batch_size_val': 1024, 'lr': 0.001, 'device': 'gpu', 'gpu_list': [0], 'dump_path': 'model_state_dumps'}


In [32]:
engine.train(epochs=2.5,report_interval=10,valid_interval=100)

Epoch 0 Starting @ 2019-06-19 19:42:43
... Iteration 1 ... Epoch 0.00 ... Loss 2.577 ... Accuracy 0.344
... Iteration 10 ... Epoch 0.00 ... Loss 1.821 ... Accuracy 0.250
... Iteration 20 ... Epoch 0.00 ... Loss 1.130 ... Accuracy 0.250
... Iteration 30 ... Epoch 0.00 ... Loss 0.998 ... Accuracy 0.531
... Iteration 40 ... Epoch 0.00 ... Loss 0.946 ... Accuracy 0.469
... Iteration 50 ... Epoch 0.00 ... Loss 1.049 ... Accuracy 0.344
... Iteration 60 ... Epoch 0.00 ... Loss 0.917 ... Accuracy 0.656
... Iteration 70 ... Epoch 0.00 ... Loss 0.934 ... Accuracy 0.469
... Iteration 80 ... Epoch 0.00 ... Loss 0.974 ... Accuracy 0.531
... Iteration 90 ... Epoch 0.00 ... Loss 0.867 ... Accuracy 0.625
... Iteration 100 ... Epoch 0.00 ... Loss 0.836 ... Accuracy 0.500
Saved checkpoint as: model_state_dumps/20190619_194243/SimpleMLP.pth
best validation loss so far!: 0.8397846817970276
Saved checkpoint as: model_state_dumps/20190619_194243/SimpleMLPBEST.pth
... Iteration 110 ... Epoch 0.00 ... Loss 0.

KeyboardInterrupt: 

In [13]:
model_MLP._get_name()

'SimpleMLP'

In [14]:
from models.simpleCNN import SimpleCNN
model_CNN=SimpleCNN(num_input_channels=38,num_classes=3)

In [15]:
def rotate_chan(x):
    return np.transpose(x,(2,0,1))

In [16]:
dset=WCH5Dataset("/fast_scratch/WatChMaL/data/IWCDmPMT/varyE/IWCDmPMT_varyE_100-1000MeV_300k_fmt.h5",val_split=0.1,test_split=0.1,transform=rotate_chan)

In [17]:
engine=Engine(model_CNN,dset,config)

Requesting GPUs. GPU list : [0]
Main GPU: cuda:0
CUDA is available
Creating a directory for run dump: model_state_dumps/20190619_194634/


In [18]:
model_CNN.parameters

<bound method Module.parameters of SimpleCNN(
  (relu): ReLU()
  (softmax): Softmax()
  (f_embed): Conv2d(38, 64, kernel_size=(1, 1), stride=(1, 1))
  (f_conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv2a): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_conv2b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv3a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_conv3b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f_max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (f_conv4): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=

In [ ]:
engine.train(epochs=5,report_interval=10,valid_interval=100)

Epoch 0 Starting @ 2019-06-19 19:46:39
... Iteration 1 ... Epoch 0.00 ... Loss 1.097 ... Accuracy 0.375
... Iteration 10 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.531
... Iteration 20 ... Epoch 0.00 ... Loss 1.076 ... Accuracy 0.469
... Iteration 30 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.406
... Iteration 40 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.406
... Iteration 50 ... Epoch 0.00 ... Loss 1.071 ... Accuracy 0.406
... Iteration 60 ... Epoch 0.00 ... Loss 1.079 ... Accuracy 0.281
... Iteration 70 ... Epoch 0.00 ... Loss 1.047 ... Accuracy 0.469
... Iteration 80 ... Epoch 0.00 ... Loss 1.088 ... Accuracy 0.375
... Iteration 90 ... Epoch 0.00 ... Loss 1.091 ... Accuracy 0.375
... Iteration 100 ... Epoch 0.00 ... Loss 1.092 ... Accuracy 0.188
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNN.pth
best validation loss so far!: 1.0666351318359375
Saved checkpoint as: model_state_dumps/20190619_194634/SimpleCNNBEST.pth
... Iteration 110 ... Epoch 0.00 ... Loss 1.